In [15]:
# from article: https://www.grace-dev.com/python-apis/strava-api/

import json
import os
import requests
import time

# Initial Settings
client_id = '99100'
client_secret = '299906f28ff8a7dd20d67270c5b1782c428b4a5e'
redirect_uri = 'http://localhost/'

# Authorization URL
request_url = f'http://www.strava.com/oauth/authorize?client_id={client_id}' \
                  f'&response_type=code&redirect_uri={redirect_uri}' \
                  f'&approval_prompt=force' \
                  f'&scope=profile:read_all,activity:read_all'

In [16]:
# User prompt showing the Authorization URL
# and asks for the code
print('Click here:', request_url)
print('Please authorize the app and copy&paste below the generated code!')
print('P.S: you can find the code in the URL')

Click here: http://www.strava.com/oauth/authorize?client_id=99100&response_type=code&redirect_uri=http://localhost/&approval_prompt=force&scope=profile:read_all,activity:read_all
Please authorize the app and copy&paste below the generated code!
P.S: you can find the code in the URL


In [17]:
code = input('Insert the code from the url: ')

# Get the access token
token = requests.post(url='https://www.strava.com/api/v3/oauth/token',
                       data={'client_id': client_id,
                             'client_secret': client_secret,
                             'code': code,
                             'grant_type': 'authorization_code'})

#Save json response as a variable
strava_token = token.json()

with open('strava_token.json', 'w') as outfile:
  json.dump(strava_token, outfile)

In [36]:
with open('strava_token.json', 'r') as token:
  data = json.load(token)

# Get the access token
access_token = data['access_token']

# Build the API url to get athlete info
athlete_url = f"https://www.strava.com/api/v3/athlete?" \
              f"access_token={access_token}"

# Get the response in json format
response = requests.get(athlete_url)
athlete = response.json()

# Print out the retrieved information
# print('RESTful API:', athlete_url)
# print('='* 5, 'ATHLETE INFO', '=' * 5)
# print('Name:', athlete['firstname'], '"' + athlete['username'] + '"', athlete['lastname'])
# print('Gender:', athlete['sex'])
# print('City:', athlete['city'], athlete['country'])
# print('Strava athlete from:', athlete['created_at'])

print(athlete)

{'id': 69122020, 'username': None, 'resource_state': 3, 'firstname': 'Justin', 'lastname': 'Long', 'bio': None, 'city': 'Clackamas', 'state': 'Oregon', 'country': None, 'sex': 'M', 'premium': True, 'summit': True, 'created_at': '2020-09-25T01:29:48Z', 'updated_at': '2022-08-05T06:57:44Z', 'badge_type_id': 1, 'weight': 68.0389, 'profile_medium': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/69122020/17007888/2/medium.jpg', 'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/69122020/17007888/2/large.jpg', 'friend': None, 'follower': None, 'blocked': False, 'can_follow': True, 'follower_count': 30, 'friend_count': 28, 'mutual_friend_count': 0, 'athlete_type': 1, 'date_preference': '%m/%d/%Y', 'measurement_preference': 'feet', 'clubs': [], 'ftp': None, 'bikes': [], 'shoes': [], 'is_winback_via_upload': False, 'is_winback_via_view': False}


In [37]:
# Read the token from the saved file
with open('strava_token.json', 'r') as token:
  data = json.load(token)

access_token = data['access_token']

# Build the API url to get activities data
activities_url = f"https://www.strava.com/api/v3/athlete/activities?" \
          f"access_token={access_token}"
print('RESTful API:', activities_url)

# Get the response in json format
response = requests.get(activities_url)
activity = response.json()[5]

# Print out the retrieved information
print('='*5, 'SINGLE ACTIVITY', '='*5)
print('Athlete:', athlete['firstname'], athlete['lastname'])
print('Name:', activity['name'])
print('Date:', activity['start_date'])
print('Disance:', activity['distance'], 'm')
print('Average Speed:', activity['average_speed'], 'm/s')
print('Max speed:', activity['max_speed'], 'm/s')
print('Moving time:', round(activity['moving_time'] / 60, 2), 'minutes')
print('Location:', activity['location_city'], 
      activity['location_state'], activity['location_country'])

RESTful API: https://www.strava.com/api/v3/athlete/activities?access_token=e0df360262eb40b719fc6b8f624401c4ec495e9b
===== SINGLE ACTIVITY =====
Athlete: Justin Long
Name: Night Run
Date: 2022-12-06T20:34:00Z
Disance: 5008.0 m
Average Speed: 3.05 m/s
Max speed: 5.36 m/s
Moving time: 27.37 minutes
Location: None None None


In [40]:
activity = response.json()[0]

miles_per_kilometer = 0.000621371

print("This activity is " + str('%.3f'%(int(activity["distance"]) * miles_per_kilometer)) + " miles")

# print(activity)

print("This activity ID is " + str(activity["id"]))

activity_id = activity["id"]

print("Activity keys: " + str(list(activity.keys())))

This activity is 9.608 miles
This activity ID is 8304940494
Activity keys: ['resource_state', 'athlete', 'name', 'distance', 'moving_time', 'elapsed_time', 'total_elevation_gain', 'type', 'sport_type', 'workout_type', 'id', 'start_date', 'start_date_local', 'timezone', 'utc_offset', 'location_city', 'location_state', 'location_country', 'achievement_count', 'kudos_count', 'comment_count', 'athlete_count', 'photo_count', 'map', 'trainer', 'commute', 'manual', 'private', 'visibility', 'flagged', 'gear_id', 'start_latlng', 'end_latlng', 'average_speed', 'max_speed', 'average_cadence', 'has_heartrate', 'average_heartrate', 'max_heartrate', 'heartrate_opt_out', 'display_hide_heartrate_option', 'elev_high', 'elev_low', 'upload_id', 'upload_id_str', 'external_id', 'from_accepted_tag', 'pr_count', 'total_photo_count', 'has_kudoed', 'suffer_score']


Idea: RunningShoesData+     (can also measure how many miles have been tracked since a given date)
- Enter a date for when you last got new running shoes
- Get all activities (from running) that have occurred past this date
    - Put their mileages into a dataframe?
    - Tally up mileages
    - Display how many miles are on this pair of shoes
    - Based on this number, when are you estimated to need new running shoes?
        - If not yet reached the cutoff, calculate the average number of miles per week (or day?)
        - See how many weeks are estimated to be needed to reach the cutoff number

In [5]:
import pandas as pd
import json
import requests

col_names = ['id','type', 'distance', 'start_date']
activities = pd.DataFrame(columns=col_names)

In [30]:
# Get all activities
with open('strava_token.json', 'r') as token:
  data = json.load(token)

# Get the access token
access_token = data['access_token']


activities_url = f"https://www.strava.com/api/v3/athlete/activities"


miles_per_meter = 0.000621371

header = {'Authorization': 'Bearer ' + access_token}

page = 1
while True:
    param = {'per_page': 200, 'page': page}
    # get page of activities from Strava
    r = requests.get(activities_url, headers=header, params=param).json()
    #print(r)

    # if no results then exit loop
    if (not r):
        print("Obtained all activities!")
        break
    
    # otherwise add new data to dataframe
    for x in range(len(r)):
        activities.loc[x + (page-1)*200,'id'] = r[x]['id']
        activities.loc[x + (page-1)*200,'type'] = r[x]['type']
        activities.loc[x + (page-1)*200,'distance'] = int(r[x]['distance']) * miles_per_meter
        activities.loc[x + (page-1)*200,'start_date'] = r[x]['start_date'][0:10]

    # increment page
    page += 1

print("Obtained data for " + str(len(activities.index)) + " activities!")

Obtained all activities!
Obtained data for 387 activities!


In [33]:
# narrow to just runs
# eliminate NaNs
# narrow to just after a certain date
# tally
activities_no_na = activities.dropna()
activities_runs = activities_no_na.loc[activities_no_na['type'] == 'Run']

new_shoes_date = '2022-03-27'
activities_since_new_shoes = activities_runs.loc[(activities_runs['start_date'] >= new_shoes_date)]


In [34]:
len(activities_since_new_shoes.index)

74

In [36]:
miles_since_new_shoes = activities_since_new_shoes['distance'].sum()
print(miles_since_new_shoes)

497.18254919800006
